In [ ]:
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166

In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
from facenet_pytorch import MTCNN
from PIL import Image
import glob


In [ ]:
def extract_frames(video_path, output_dir, k=16):
    """
    Extract k frames from the video at regular intervals.
    If the video has fewer frames, oversample by duplicating frames.
    """
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print(f"Skipping {video_path}: No frames detected.")
        cap.release()
        return

    # Calculate frame indices for sampling
    frame_indices = (
        np.tile(np.arange(total_frames), k // total_frames + 1)[:k]  # Oversample if frames are fewer
        if total_frames < k
        else np.linspace(0, total_frames - 1, k, dtype=int)  # Sample evenly
    )

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame_path = os.path.join(output_dir, f"{idx:04d}.jpg")
            cv2.imwrite(frame_path, frame)

    cap.release()
    print(f"Extracted frames from {video_path}")


In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as transforms  # Add this import

# Initialize MTCNN for face cropping
mtcnn = MTCNN(image_size=224, margin=20)

# Define the process and crop frames function
def process_and_crop_frames(video_path, cropped_dir, k=16):
    """
    Extract k frames from the video at regular intervals and save only cropped faces.
    Args:
        video_path (str): Path to the video file.
        cropped_dir (str): Directory to save cropped face images.
        k (int): Number of frames to extract per video.
    """
    os.makedirs(cropped_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print(f"Skipping {video_path}: No frames detected.")
        cap.release()
        return

    # Calculate frame indices for sampling
    frame_indices = (
        np.tile(np.arange(total_frames), k // total_frames + 1)[:k]
        if total_frames < k
        else np.linspace(0, total_frames - 1, k, dtype=int)
    )

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            # Convert frame to RGB and detect/crop face
            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            faces = mtcnn(img)  # mtcnn returns a list of face tensors and corresponding probabilities

            if faces is not None:
                for i, face in enumerate(faces):
                    # Convert the face tensor to a PIL Image
                    face_image = transforms.ToPILImage()(face)

                    # Save the cropped face image
                    cropped_path = os.path.join(cropped_dir, f"{os.path.basename(video_path).split('.')[0]}_{idx:04d}_{i}.jpg")
                    face_image.save(cropped_path, quality=85)
            else:
                print(f"No face detected in frame {idx} of video {video_path}")

    cap.release()
    print(f"Processed and cropped frames from {video_path}")


In [ ]:
def process_videos_in_batches(video_paths, cropped_base_dir, k=16, batch_size=50):
    """
    Processes videos in batches, extracting frames and cropping faces directly.
    Args:
        video_paths (list): List of video file paths to process.
        cropped_base_dir (str): Base directory for saving cropped faces.
        k (int): Number of frames to extract per video.
        batch_size (int): Number of videos to process in a single batch.
    """
    total_videos = len(video_paths)
    for start_idx in range(0, total_videos, batch_size):
        batch_videos = video_paths[start_idx:start_idx + batch_size]
        print(f"Processing batch: {start_idx + 1}-{min(start_idx + batch_size, total_videos)}")

        for video in batch_videos:
            video_name = os.path.basename(video).split('.')[0]
            cropped_dir = os.path.join(cropped_base_dir, video_name)

            # Extract and crop frames directly
            process_and_crop_frames(video, cropped_dir, k=k)

        print(f"Completed batch: {start_idx + 1}-{min(start_idx + batch_size, total_videos)}\n")


In [ ]:
# Define paths
real_videos = glob.glob('drive/MyDrive/final_dataset/fake/*.mp4')  # Real videos path
#fake_videos = glob.glob('drive/MyDrive/final_dataset/fake/*.mp4')  # Fake videos path

# Directories for cropped faces
cropped_base_dir_real = 'drive/MyDrive/final_dataset/cropped_frames/fake'
#cropped_base_dir_fake = 'drive/MyDrive/final_dataset/cropped_frames/fake'

# Process real videos
process_videos_in_batches(real_videos, cropped_base_dir_real, k=16, batch_size=50)

# Process fake videos
#process_videos_in_batches(fake_videos, cropped_base_dir_fake, k=16, batch_size=50)


Processing batch: 889-938
Processed and cropped frames from drive/MyDrive/final_dataset/real/id61_0009.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/id61_0008.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_abarnvbtwb.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_aelfnikyqj.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_afoovlsmtx.mp4
No face detected in frame 0 of video drive/MyDrive/final_dataset/real/0_agrmhtjdlk.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_agrmhtjdlk.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_ahqqqilsxt.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_ajqslcypsw.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_anpuvshzoo.mp4
Processed and cropped frames from drive/MyDrive/final_dataset/real/0_asaxgevnnp.mp4
Processed and cropped frames from drive/MyDrive/final